In [1]:
!pip install tensorflow

  Using cached tensorflow-2.12.0-cp39-cp39-win_amd64.whl (1.9 kB)
  Using cached tensorflow_intel-2.12.0-cp39-cp39-win_amd64.whl (272.8 MB)
  Using cached opt_einsum-3.3.0-py3-none-any.whl (65 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
  Using cached protobuf-4.23.3-cp39-cp39-win_amd64.whl (422 kB)
  Using cached absl_py-1.4.0-py3-none-any.whl (126 kB)
  Using cached termcolor-2.3.0-py3-none-any.whl (6.9 kB)
  Using cached grpcio-1.54.2-cp39-cp39-win_amd64.whl (4.1 MB)
  Using cached tensorflow_estimator-2.12.0-py2.py3-none-any.whl (440 kB)
  Using cached tensorboard-2.12.3-py3-none-any.whl (5.6 MB)
  Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
  Using cached keras-2.12.0-py2.py3-none-any.whl (1.7 MB)
  Using cached libclang-16.0.0-py2.py3-none-win_amd64.whl (24.4 MB)
  Using cached tensorflow_io_gcs_filesystem-0.31.0-cp39-cp39-win_amd64.whl (1.5 MB)
  Using cached gast-0.4.0-py3-none-any.whl (9.8 kB)
  Using cached jax-0.4.12-py3-none-any.whl
  Using

In [2]:
!pip install tensorflow_datasets

  Created wheel for promise: filename=promise-2.3-py3-none-any.whl size=21502 sha256=8c592a39fd5da05d5500202c06a426731947cd7b412233d3e09036b5e1ae42cc
  Stored in directory: c:\users\raghav_agarwal\appdata\local\pip\cache\wheels\e1\e8\83\ddea66100678d139b14bc87692ece57c6a2a937956d2532608
Successfully built promise


In [3]:
import numpy as np
import tensorflow as tf

import tensorflow_datasets as tfds

In [6]:
mnist_dataset, mnist_info = tfds.load(name='mnist', with_info=True, as_supervised=True)

In [7]:
print(mnist_info)

tfds.core.DatasetInfo(
    name='mnist',
    full_name='mnist/3.0.1',
    description="""
    The MNIST database of handwritten digits.
    """,
    homepage='http://yann.lecun.com/exdb/mnist/',
    data_path='C:\\Users\\raghav_agarwal\\tensorflow_datasets\\mnist\\3.0.1',
    file_format=tfrecord,
    download_size=11.06 MiB,
    dataset_size=21.00 MiB,
    features=FeaturesDict({
        'image': Image(shape=(28, 28, 1), dtype=uint8),
        'label': ClassLabel(shape=(), dtype=int64, num_classes=10),
    }),
    supervised_keys=('image', 'label'),
    disable_shuffling=False,
    splits={
        'test': <SplitInfo num_examples=10000, num_shards=1>,
        'train': <SplitInfo num_examples=60000, num_shards=1>,
    },
    citation="""@article{lecun2010mnist,
      title={MNIST handwritten digit database},
      author={LeCun, Yann and Cortes, Corinna and Burges, CJ},
      journal={ATT Labs [Online]. Available: http://yann.lecun.com/exdb/mnist},
      volume={2},
      year={2010}
  

## Pre-processing the data

In [60]:
mnist_train, mnist_test = mnist_dataset['train'], mnist_dataset['test']

num_validation_samples = 0.1 * mnist_info.splits['train'].num_examples
num_validation_samples = tf.cast(num_validation_samples, tf.int64)

num_test_samples = mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_test_samples, tf.int64)

print(num_test_samples, type(num_test_samples))


# SCALING THE DATA
# This fn will always take image and label as input and will always return image and label as output
def scale(image, label):
    # need to convert image to float
    image = tf.cast(image, tf.float32)
    
    # dividing each pixel value by 255 to make it b/w 0 and 1.
    # since we need float values dividing by (255.).
    image /= 255.
    
    return image, label

scaled_train_and_validation_data = mnist_train.map(scale)
test_data = mnist_test.map(scale)


# SHUFFLING THE DATA

BUFFER_SIZE = 1000

shuffled_train_and_validation_data = scaled_train_and_validation_data.shuffle(BUFFER_SIZE)
validation_data = shuffled_train_and_validation_data.take(num_validation_samples)
train_data = shuffled_train_and_validation_data.skip(num_validation_samples)


# CREATING BATCHES FOR STOCHASTIC GRADIENT DESCENT
# This will add a new column which will help in detecting batch.

BATCH_SIZE = 100

train_data = train_data.batch(BATCH_SIZE)
validation_data = validation_data.batch(num_validation_samples)
test_data = test_data.batch(num_test_samples)

validation_inputs, validation_targets = next(iter(validation_data))
    

tf.Tensor(10000, shape=(), dtype=int64) <class 'tensorflow.python.framework.ops.EagerTensor'>


## Outline the Model

In [74]:
# Input layer consists of 784 nodes (An image here is of 28*28 pixels)
# Output size is of 10 (The image can contain 10 digits -> 0,1,2,3,4,5,6,7,8,9)
# Hidden layer nodes is a hyperparamter which we can define.

input_size = 784
output_size = 10
hidden_layer_size = 200

model = tf.keras.Sequential([
    # This is input layer we need to flatten 28x28 image into a vector
    tf.keras.layers.Flatten(input_shape=(28,28,1)),
    
    # This is a hidden layer, with activation fn as relu
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    
    # This is a hidden layer, with activation fn as relu
    tf.keras.layers.Dense(hidden_layer_size, activation='tanh'),
    
    # This is a hidden layer, with activation fn as relu
    tf.keras.layers.Dense(hidden_layer_size, activation='tanh'),
    
    # This is a hidden layer, with activation fn as relu
    tf.keras.layers.Dense(hidden_layer_size, activation='tanh'),
    
    # This is a hidden layer, with activation fn as relu
    tf.keras.layers.Dense(hidden_layer_size, activation='tanh'),
    
    # This is output layer, since it is a classification requirement, we have used softmax fn to give probabilities in output layer
    tf.keras.layers.Dense(output_size, activation='softmax'),
])

## Choosing theoptimizer and loss function

In [75]:
# The compile method configures the model for training
# Optimizer - Adaptive Moment Estimation or Adam
# For classifiers - loss fn would be Cross entropy
# There are 3 types of cross entropy in tensorflow
# 1. Binary_crossentropy - Expects data in binary encoding
# 2. Categorical_crossentropy - expects one-hot encoded targets
# 3. Sparse_categorical_crossentropy - applies one hot encoding by itself
# We can also include some metrics for tracking purpose like accuracy

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# For custom learning rate
# custom_optimizer = tf.keras.optimizers.Adam(learning_rate=0.02)
# model.compile(optimizer=custom_optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

## Training the Model

In [77]:
NUM_EPOCHS = 5

# 1. At the beginning of each epoch, training loss will be 0.
# 2. The algo will iterate over preset no of batches all from train_data.
# 3. The weights and biases will be updated as many times as batches.
# 4. We will get a value of loss fn indicating how training is gng.
# 5. We will also get training accuracy
# 6. At the end of each epoch, the algo will forward propagate the whole validation set.

# Validation accuracy - TRue accuracy of our model
# Check Validation loss and accuracy to avoid overfitting and early stopping the model.
# Accuracy represent in how many cases predicted output equals to actual output
model.fit(train_data, epochs = NUM_EPOCHS, validation_data = (validation_inputs, validation_targets), verbose = 2)

Epoch 1/5
540/540 - 4s - loss: 0.0230 - accuracy: 0.9925 - val_loss: 0.1210 - val_accuracy: 0.9718 - 4s/epoch - 7ms/step
Epoch 2/5
540/540 - 3s - loss: 0.0217 - accuracy: 0.9934 - val_loss: 0.1155 - val_accuracy: 0.9738 - 3s/epoch - 6ms/step
Epoch 3/5
540/540 - 4s - loss: 0.0200 - accuracy: 0.9937 - val_loss: 0.1078 - val_accuracy: 0.9750 - 4s/epoch - 7ms/step
Epoch 4/5
540/540 - 3s - loss: 0.0193 - accuracy: 0.9937 - val_loss: 0.0930 - val_accuracy: 0.9795 - 3s/epoch - 6ms/step
Epoch 5/5
540/540 - 3s - loss: 0.0201 - accuracy: 0.9934 - val_loss: 0.1010 - val_accuracy: 0.9770 - 3s/epoch - 6ms/step


## Testing the Model

In [78]:
test_loss, test_accuracy = model.evaluate(test_data)
print('Test loss: {0:.2f}. Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100.))

1/1 [==============================] - 1s 816ms/step - loss: 0.1003 - accuracy: 0.9784
Test loss: 0.10. Test accuracy: 97.84%
